##### Stage 2: Send data to AWS

**For required libraries to run this notebook read the section *Pre-requisites* located in ```Docs/Project/Charter.md``` file for detailed information.**

In [1]:
""" If needed it can be installed with these commented lines """

#!pip3 install aws-iot
#!pip3 install awscrt
#!pip3 install AWSIoTPythonSDK
#!pip3 install awsiot
#!pip3 install awsiotsdk

' If needed it can be installed with these commented lines '

In [2]:
# importing the necessary libraries
from awscrt import io, mqtt, auth, http
from awsiot import mqtt_connection_builder
import re
import time as t
import json


# setting the MQTT connection parameters
ENDPOINT = "[OMITTED FOR SECURITY PURPOSES]"
CLIEND_ID = "[OMITTED FOR SECURITY PURPOSES]"
PATH_TO_CERTIFICATE = "[OMITTED FOR SECURITY PURPOSES]"
PATH_TO_PRIVATE_KEY = "[OMITTED FOR SECURITY PURPOSES]"
PATH_TO_AMAZON_ROOT_CA_1 = "[OMITTED FOR SECURITY PURPOSES]"
TOPIC = "[OMITTED FOR SECURITY PURPOSES]"


# function to read the CSV file
def read_last_entries(current_line=0):
    
    with open("../../Docs/Assets/room.csv", "r") as entries:
        entries.seek(current_line)
        for line in entries:
            yield line.split(',')

# function to filter content
def filter_content(line, pattern="Timestamp, Temperature_Celsius, Relative_Humidity, Date, Season, Wind, ApparentTemperature:"):
    if re.search(pattern, line):
        value = re.search(r"\d+\.\d+", line).group(0)
        return float(value)
    return None


# opening the connection, sending the parsed JSON data and closing the connection
if __name__ == "__main__":
    read_from_line = 0
    event_loop_group = io.EventLoopGroup(1)
    host_resolver = io.DefaultHostResolver(event_loop_group)
    client_bootstrap = io.ClientBootstrap(event_loop_group,  host_resolver)
    mqtt_connection = mqtt_connection_builder.mtls_from_path(
        endpoint=ENDPOINT,
        cert_filepath=PATH_TO_CERTIFICATE,
        pri_key_filepath=PATH_TO_PRIVATE_KEY,
        ca_filepath=PATH_TO_AMAZON_ROOT_CA_1,
        client_bootstrap=client_bootstrap,
        client_id=CLIEND_ID,
        clean_session=False,
        keep_alive_secs=6
    )
    connection_future = mqtt_connection.connect()
    connection_future.result()
    for idx, line in enumerate(read_last_entries(current_line=read_from_line)):
        if idx == 0:
            continue
        timestamp,temperature_celsius,relative_humidity, date, season, wind, apparent_temperature = line

        message = {"timestamp":timestamp, "temperature_celsius" :  temperature_celsius, "relative_humidity" : relative_humidity, "date":date, "season":season, "wind":wind, "apparent_temperature":apparent_temperature}
        print("Sending message to Aws Iot Core: {}".format(message))
        mqtt_connection.publish(topic=TOPIC,
                                            payload=json.dumps(message),
                                            qos=mqtt.QoS.AT_LEAST_ONCE)
        t.sleep(.1)
    read_from_line = idx
    disconnect_future = mqtt_connection.disconnect()
    disconnect_future.result()

Sending message to Aws Iot Core: {'timestamp': '2021-01-09 12:00:00', 'temperature_celsius': '21.0', 'relative_humidity': '35.0', 'date': '2021-01-09 12:00:00', 'season': 'winter', 'wind': '0.15', 'apparent_temperature': '22.1\n'}
Sending message to Aws Iot Core: {'timestamp': '2021-01-09 18:00:00', 'temperature_celsius': '23.0', 'relative_humidity': '25.0', 'date': '2021-01-09 18:00:00', 'season': 'winter', 'wind': '0.15', 'apparent_temperature': '23.4\n'}
Sending message to Aws Iot Core: {'timestamp': '2021-01-10 00:00:00', 'temperature_celsius': '21.0', 'relative_humidity': '26.0', 'date': '2021-01-10 00:00:00', 'season': 'winter', 'wind': '0.15', 'apparent_temperature': '21.2\n'}
Sending message to Aws Iot Core: {'timestamp': '2021-01-10 06:00:00', 'temperature_celsius': '21.0', 'relative_humidity': '27.0', 'date': '2021-01-10 06:00:00', 'season': 'winter', 'wind': '0.15', 'apparent_temperature': '21.3\n'}
Sending message to Aws Iot Core: {'timestamp': '2021-01-10 12:00:00', 'tempe